In [1]:
import pandas as pd
import librosa
import tensorflow as tf
import numpy as np
from tqdm import tqdm


In [2]:

# Reading the CSV file
df = pd.read_csv('cv-valid-train.csv')
df = df[~df['gender'].isna()]

BASE_DIR = 'cv-valid-train/'
X_train, y_train = [], []

for i in tqdm(df.index):
    file = BASE_DIR + df.loc[i, 'filename']
    wf, sr = librosa.load(file)
    mfcc_wf = librosa.feature.mfcc(y=wf, sr=sr)
    b = tf.keras.utils.pad_sequences(mfcc_wf, padding='post', maxlen=200)
    X_train.append(b)
    if df.loc[i, 'gender'] == 'male':
        y_train.append(1)
    else:
        y_train.append(0)

X_train = np.array(X_train)
y_train = np.array(y_train)


  0%|          | 300/74059 [00:07<29:15, 42.02it/s] 


KeyboardInterrupt: 

In [14]:
df = pd.read_csv('cv-valid-train.csv')
df = df[~df['gender'].isna()]

In [19]:
df.shape

(74059, 8)

In [22]:
df1=df.copy()

In [43]:
df

,filename,text,up_votes,down_votes,age,gender,accent,duration
5,cv-valid-train/sample-000005.mp3,a shepherd may like to travel but he should ne...,1,0,twenties,female,us,NaN
8,cv-valid-train/sample-000008.mp3,put jackie right on the staff,3,0,seventies,male,us,NaN
13,cv-valid-train/sample-000013.mp3,but he had found a guide and didn't want to mi...,1,0,thirties,female,us,NaN
14,cv-valid-train/sample-000014.mp3,as they began to decorate the hallway a silhou...,1,0,sixties,male,england,NaN
19,cv-valid-train/sample-000019.mp3,then they got ahold of some dough and went goofy,1,0,fifties,male,australia,NaN
...,...,...,...,...,...,...,...,...
195766,cv-valid-train/sample-195766.mp3,but before i go i want to tell you a little story,1,0,fourties,male,england,NaN
195767,cv-valid-train/sample-195767.mp3,down below in the darkness were hundreds of pe...,3,0,thirties,female,us,NaN
195770,cv-valid-train/sample-195770.mp3,he heard a muffled grating sound and saw the b...,4,0,twenties,male,england,NaN
195771,cv-valid-train/sample-195771.mp3,the englishman said nothing,1,0,thirties,male,england,NaN


In [33]:
#df1=df1.drop(['text'],axis=1)
df1=df1.drop(['up_votes'],axis=1)
df1=df1.drop(['down_votes'],axis=1)

In [35]:
df1=df1.drop(['duration'],axis=1)

In [36]:
df1

,filename,age,gender,accent
5,cv-valid-train/sample-000005.mp3,twenties,female,us
8,cv-valid-train/sample-000008.mp3,seventies,male,us
13,cv-valid-train/sample-000013.mp3,thirties,female,us
14,cv-valid-train/sample-000014.mp3,sixties,male,england
19,cv-valid-train/sample-000019.mp3,fifties,male,australia
...,...,...,...,...
195766,cv-valid-train/sample-195766.mp3,fourties,male,england
195767,cv-valid-train/sample-195767.mp3,thirties,female,us
195770,cv-valid-train/sample-195770.mp3,twenties,male,england
195771,cv-valid-train/sample-195771.mp3,thirties,male,england


In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(20, 200,)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=64, validation_split=0.2)


NameError: name 'X_train' is not defined

In [4]:
model.save('trained_model')


INFO:tensorflow:Assets written to: trained_model\assets


INFO:tensorflow:Assets written to: trained_model\assets


In [3]:
loaded_model = tf.keras.models.load_model('trained_model')


In [2]:
import pyaudio
import wave


In [6]:

def record_audio():
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "output.wav"

    audio = pyaudio.PyAudio()

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK,
                        input_device_index=3)

    print("Recording...")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    return WAVE_OUTPUT_FILENAME

audio_file = record_audio()

wf, sr = librosa.load(audio_file)
mfcc_wf = librosa.feature.mfcc(y=wf, sr=sr)
b = tf.keras.utils.pad_sequences(mfcc_wf, padding='post', maxlen=200)

prediction = loaded_model.predict(np.array([b]))

if prediction[0][0] > 0.5:
    print("Predicted gender: Male")
else:
    print("Predicted gender: Female")


Recording...
Finished recording.
1/1 [==============================] - 0s 23ms/step
Predicted gender: Male
